Get the Cookies and headers More like a people.

In [33]:
from seleniumwire import webdriver  
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

# - step1 selenium 上網登入 ee-class
url = "https://military.ncu.edu.tw/index.php" # = ee-class url

# : user agent setting
ua = UserAgent()
user_agent = ua.random

# : option setting
chrome_opt = Options()
chrome_opt.add_argument("start-maximized")  # 設定瀏覽器啟動時開啟視窗最大化，提供較大的視野和操作空間
chrome_opt.add_argument("--disable-proxy-certificate-handler")  # 禁用代理伺服器的證書處理，避免因證書問題而影響網頁載入
chrome_opt.add_argument("--disable-content-security-policy")  # 禁用內容安全政策(CSP)，允許加載非同源的腳本，有助於測試開發
chrome_opt.add_argument("--ignore-certificate-errors")  # 忽略證書錯誤，有助於訪問自簽名證書的開發環境或測試站點
chrome_opt.add_argument("--incognito")  # 啟動無痕(隱身)模式，不會保存瀏覽歷史、Cookies等資料，有助於保護隱私和測試

# : driver setting 
driver = webdriver.Chrome(options = chrome_opt) # => insert the option to the driver (can be overridden)
driver.delete_all_cookies() # 防止污染
driver.get(url)
sleep(2)

# @ Action.
nav_link = driver.find_element(By.ID, "nav-link")
driver.execute_script("arguments[0].click();", nav_link)

# : Get the Cookies & Headers
current_login_cookie = driver.get_cookies()
current_login_requests = driver.requests

headers = {}
for request in current_login_requests:
    if request.url == url:
        headers = {k: v for k, v in request.headers.items()}

print("Cookies:", current_login_cookie)
print("Headers:", headers)

sleep(5)
driver.quit()

Cookies: [{'domain': '.ncu.edu.tw', 'expiry': 1745064614, 'httpOnly': False, 'name': '_ga_CLHSRC7YPQ', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1710504614.1.0.1710504614.0.0.0'}, {'domain': '.ncu.edu.tw', 'expiry': 1745064614, 'httpOnly': False, 'name': '_ga', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GA1.1.574819090.1710504614'}, {'domain': 'military.ncu.edu.tw', 'httpOnly': False, 'name': 'PHPSESSID', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '1d2o1d2fj85bli4593tfpdmo11'}]
Headers: {'Host': 'military.ncu.edu.tw', 'Connection': 'keep-alive', 'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': '"macOS"', 'Upgrade-Insecure-Requests': '1', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/we

In [35]:
for request in current_login_requests:
    if request.url == url:
        headers = {k: v for k, v in request.headers.items()}
        
headers

{'Host': 'military.ncu.edu.tw',
 'Connection': 'keep-alive',
 'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
 'sec-ch-ua-mobile': '?0',
 'sec-ch-ua-platform': '"macOS"',
 'Upgrade-Insecure-Requests': '1',
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
 'Sec-Fetch-Site': 'same-origin',
 'Sec-Fetch-Mode': 'navigate',
 'Sec-Fetch-Dest': 'document',
 'Referer': 'https://military.ncu.edu.tw/epaper/index.php',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'zh-TW,zh;q=0.9',
 'Cookie': 'PHPSESSID=1d2o1d2fj85bli4593tfpdmo11'}

Request the website Content

In [36]:
import requests
from bs4 import BeautifulSoup
import re

try:
    with requests.Session() as session: 
    # ~ Session can update the performance of the server. 
    
        # : Update the Cookies & Headers from driver
        for cookie in current_login_cookie:
            session.cookies.set(cookie['name'], cookie['value'])
        session.headers.update(headers)
    
        # :  Get the response of the website.
        resp = session.get(url)
        
        # @ Check the response and Parser the response to HTML.
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # @ Get the elements of the site.
        title = soup.title.get_text() if soup.title else ''
        
        content = soup.get_text().strip()
        content = re.sub(r'\n+', '\n', content)
        
        links = []
        all_links = soup.find_all(href=True)
        for link in all_links:
            href = link.get('href') 
            text = link.get_text().strip()

            if href:
                
                # : Get the Link and Href from the link
                if "http" not in href: 
                    href = f"{url}/{href}" if href[0] != "/" else f"{url}{href}" # = href that include the relative path
                
                if text == "": 
                    text = link.get('title','').strip()
                
                links.append((text,href))
                
    print(title, content, links)
            
except requests.HTTPError as e:
    print('HTTP error', e.response.status_code)
except requests.RequestException as e:
    print('Request error:', e)
except Exception as e:
    print('', '', [])

國立中央大學生活輔導組 ﻿

國立中央大學生活輔導組
回頁首
中大首頁
學生事務處
 管理者登入
關於生輔組
組織介紹
成員介紹
地理位置
聯絡我們
校安中心
藥物濫用
折抵役期
交通安全
品德教育
校園安全
災害防救
國軍招募
智慧財產
防制霸凌
防制詐騙
校安中心FB
生活輔導
學生兵役
獎懲／操行
學生請假
研究生獎助學金
獎學金
學雜費減免
弱勢助學計畫-弱勢助學金
弱勢助學計畫-生活助學金
就學貸款
急難救助金
校內工讀
Information for International Students
遺失物招領
外宿資訊
表單下載
相關連結
緊急應變樓層平面圖
學生藥物防治資源網
社會安全網 關懷e起來
國防部
國軍人才招募中心
165全民防詐騙網
內政部役政署
財團法人法律扶助基金會
消防影音新聞台
交通安全入口網
校園霸凌防治網
崔媽媽基金會
法治教育學生手冊下載區
智慧財產局
中央大學研究發展處
衛福部疾管署
相關法規
快速連結
兵役緩徵查詢
常見Q&A
獎助學金管理系統
就學補助系統
活動回顧
校園安全地圖

            ﻿
提醒各位同學使用瓦斯熱水器，須注意室內保持空氣流通
預防七大熱門詐騙請參閱校園安全公告
 最新公告
查詢類別消息
校園安全
交通安全
防藥物濫用
軍訓通訊
遺失物協尋
品德教育
智慧財產
獎助學金
學生兵役
就學貸款
學雜費減免
急難救助
弱勢助學
學生請假
學生獎懲
其他
公告日期
公告類別
公告標題
2024-03-14
交通安全
交通安全報您知第0029期:「搭乘大眾交通工具應注意安全事項」
2024-02-29
交通安全
中華民國汽車安全協會「學生總動員」交通安全大競賽─113年度全國汽車交通安全海報創作大賽
2024-02-29
弱勢助學
§§自113年度(本學期)起~「大專校院弱勢學生助學計畫-校外租屋租金補貼」申請方式變更
2024-02-20
其他
第五屆百花川詩獎徵稿
2024-02-20
交通安全
113年交通部機車考照補助
2024-03-01
學生兵役
「國防部全球資訊網」成立1年期義務役專區
2024-02-22
急難救助
公告「財團法人台北行天宮急難濟助辦法」
2024-02-21
校園安全
§§ 開學了~~請同學於2/28日前上Portal更新校外租屋地址資料，以完成註冊程序(未完成者無

In [ ]:
import collections
dq = collections.deque([1,2,3,4,5])
while dq:
    print(dq.popleft())
    